In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
pip install emoji

     |████████████████████████████████| 51kB 2.7MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42176 sha256=a7d6d9fab094ef14eef3a8ada5f9e81b2b8540c14ddd6e0e4a0a95f28a0e1f31
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


**LSTM with Glove word embedding I got 67.5% accuracy on Test data.**

In [3]:
import nltk
import pandas as pd
import re
import string
import emoji
import itertools
from keras.layers import Conv1D, MaxPooling1D, Dropout, Dense
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer 
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
data = pd.read_csv('/content/drive/My Drive/train.txt')
train_dataset = pd.DataFrame(data, columns= ['tweet_id','sentiment','tweet_text'])

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
def load_dict_contractions():
    
    return {
        "ain't":"is not",
        "lmao" :"laughing out loud",
        "lol":"laughing out loud",
        "aint":"is not",
        "ain":"is not",
        "amn't":"am not",
        "amnt":"am not",
        "amn":"am not",
        "aren't":"are not",
        "arent":"are not",
        "aren":"are not",
        "can't":"cannot",
        "cant":"cannot",
        "'cause":"because",
        "couldn't":"could not",
        "couldnt":"could not",
        "couldn":"could not",
        "couldn't've":"could not have",
        "couldnt've":"could not have",
        "couldn've":"could not have",        
        "could've":"could have",
        "couldve":"could have",
        "daren't":"dare not",
        "daresn't":"dare not",
        "dasn't":"dare not",
        "didn't":"did not",
        "didnt":"did not",
        "didn":"did not",
        "doesn't":"does not",
        "doesnt":"does not",
        "doesn":"does not",
        "don't":"do not",
        "dont":"do not",
        "e'er":"ever",
        "em":"them",
        "everyone's":"everyone is",
        "finna":"fixing to",
        "gimme":"give me",
        "gonna":"going to",
        "gon't":"go not",
        "gotta":"got to",
        "hadn't":"had not",
        "hadnt":"had not",
        "hadn":"had not",
        "hasn't":"has not",
        "hasnt":"has not",
        "hasn":"has not",
        "haven't":"have not",
        "havent":"have not",
        "haven":"have not",
        "he'd":"he would",
        "he'll":"he will",
        "he's":"he is",
        "he've":"he have",
        "how'd":"how would",
        "how'll":"how will",
        "how're":"how are",
        "how's":"how is",
        "I'd":"I would",
        "I'll":"I will",
        "I'm":"I am",
        "I'm'a":"I am about to",
        "I'm'o":"I am going to",
        "isn't":"is not",
        "it'd":"it would",
        "it'll":"it will",
        "it's":"it is",
        "I've":"I have",
        "kinda":"kind of",
        "let's":"let us",
        "mayn't":"may not",
        "maynt":"may not",
        "may've":"may have",
        "mightn't":"might not",
        "might've":"might have",
        "mustn't":"must not",
        "mustn't've":"must not have",
        "must've":"must have",
        "needn't":"need not",
        "neednt":"need not",
        "ne'er":"never",
        "o'":"of",
        "o'er":"over",
        "ol'":"old",
        "oughtn't":"ought not",
        "shalln't":"shall not",
        "shan't":"shall not",
        "she'd":"she would",
        "she'll":"she will",
        "she's":"she is",
        "shouldn't":"should not",
        "shouldn't've":"should not have",
        "should've":"should have",
        "somebody's":"somebody is",
        "someone's":"someone is",
        "something's":"something is",
        "that'd":"that would",
        "that'll":"that will",
        "that're":"that are",
        "that's":"that is",
        "there'd":"there would",
        "there'll":"there will",
        "there're":"there are",
        "there's":"there is",
        "these're":"these are",
        "they'd":"they would",
        "they'll":"they will",
        "they're":"they are",
        "they've":"they have",
        "this's":"this is",
        "those're":"those are",
        "'tis":"it is",
        "'twas":"it was",
        "wanna":"want to",
        "wasn't":"was not",
        "we'd":"we would",
        "we'd've":"we would have",
        "we'll":"we will",
        "we're":"we are",
        "weren't":"were not",
        "we've":"we have",
        "what'd":"what did",
        "what'll":"what will",
        "what're":"what are",
        "what's":"what is",
        "what've":"what have",
        "when's":"when is",
        "where'd":"where did",
        "where're":"where are",
        "where's":"where is",
        "where've":"where have",
        "which's":"which is",
        "who'd":"who would",
        "who'd've":"who would have",
        "who'll":"who will",
        "who're":"who are",
        "who's":"who is",
        "who've":"who have",
        "why'd":"why did",
        "why're":"why are",
        "why's":"why is",
        "won't":"will not",
        "wont":"will not",
        "wouldn't":"would not",
        "wouldnt":"would not",
        "would've":"would have",
        "y'all":"you all",
        "you'd":"you would",
        "you'll":"you will",
        "you're":"you are",
        "you've":"you have",
        "Whatcha":"What are you",
        "luv":"love",
        "sux":"sucks"
        }

In [5]:
def load_dict_smileys():
    
    return {
        ":‑)":"smiley",
        ":-]":"smiley",
        ":-3":"smiley",
        ":->":"smiley",
        "8-)":"smiley",
        ":-}":"smiley",
        ":)":"smiley",
        ":]":"smiley",
        ":3":"smiley",
        ":>":"smiley",
        "8)":"smiley",
        ":}":"smiley",
        ":o)":"smiley",
        ":c)":"smiley",
        ":^)":"smiley",
        "=]":"smiley",
        "=)":"smiley",
        ":-))":"smiley",
        ":‑D":"smiley",
        "8‑D":"smiley",
        "x‑D":"smiley",
        "X‑D":"smiley",
        ":D":"smiley",
        "8D":"smiley",
        "xD":"smiley",
        "XD":"smiley",
        ":‑(":"sad",
        ":‑c":"sad",
        ":‑<":"sad",
        ":‑[":"sad",
        ":(":"sad",
        ":c":"sad",
        ":<":"sad",
        ":[":"sad",
        ":-||":"sad",
        ">:[":"sad",
        ":{":"sad",
        ":@":"sad",
        ">:(":"sad",
        ":'‑(":"sad",
        ":'(":"sad",
        ":‑P":"playful",
        "X‑P":"playful",
        "x‑p":"playful",
        ":‑p":"playful",
        ":‑Þ":"playful",
        ":‑þ":"playful",
        ":‑b":"playful",
        ":P":"playful",
        "XP":"playful",
        "xp":"playful",
        ":p":"playful",
        ":Þ":"playful",
        ":þ":"playful",
        ":b":"playful",
        "<3":"love"
        }

In [7]:
def clean_text_r1(text):
    text= text.lower()
    
    text = re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", " ", text)  #remove mentions and hashtag
    text= re.sub('\[.*?\]','',text) #remove punctuations
    #print(text)
    text =re.sub("(\w+:\/\/\S+)", " ", text) ##remove url
    #print(text)

    text = re.sub('\w*\d','',text) #remove digits after words or in between words
    #print(text)
    CONTRACTIONS = load_dict_contractions()
    text = text.replace("’","'")
    words = text.split()
    reformed = [CONTRACTIONS[word] if word in CONTRACTIONS else word for word in words]
    text = " ".join(reformed)
    
    text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))
    #emojis
    SMILEY = load_dict_smileys()  
    words = text.split()
    reformed = [SMILEY[word] if word in SMILEY else word for word in words]
    text = " ".join(reformed)
    #print(text)
    text = emoji.demojize(text)
    text = text.replace(":"," ")
    text = ' '.join(text.split())
    #print(text)
    text= re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    #print(text)
    return text


def remove_one_or_two_alpha(tweet):  
  for word in tweet:
    if(len(word) < 3):
      tweet.remove(word)
  return tweet



In [9]:

# Text Preprocessing
def cleaning_part2(dataset):
    wordNetLemmatizer = WordNetLemmatizer()
    porterStemmer = PorterStemmer()
    data_clean = dataset.apply(lambda x: clean_text_r1(x))

    data_clean =data_clean.apply(lambda tweet: [wordNetLemmatizer.lemmatize(word) for word in nltk.word_tokenize(tweet)])
    data_clean = data_clean.apply(lambda tweet: [word for word in tweet if word not in stopwords.words('english')])
    data_clean = data_clean.apply(lambda tweet : remove_one_or_two_alpha(tweet))
    data_clean = data_clean.apply(lambda tweet: ' '.join(tweet))
    data_clean_list =[]
    for tweet in data_clean:
        data_clean_list.append(tweet);
    return data_clean_list

def label(senti):
  if(senti == "negative"):
    senti =0
  if(senti == "neutral"):
    senti= 1
  if(senti == "positive"):
    senti = 2
  return senti

reviews_train_clean = cleaning_part2(train_dataset['tweet_text'])
reviews_labels =[]
for sentiment in train_dataset['sentiment']:
  sentiment = label(sentiment)
  reviews_labels.append(sentiment)

#uncomment it if you want to Check cleaning of texts by saving them into another file.  
'''
# Create DataFrame 
dfn = pd.DataFrame()
dfn['tweet_orig'] = train_dataset['tweet_text']
dfn['tweet_new'] = reviews_train_clean

dfn.to_csv('/content/drive/My Drive/lrtext1.csv', header=True, index=False) 


reviews_train_clean = cleaningPart2(train_dataset['tweet_text'])
  '''
    


"\n# Create DataFrame \ndfn = pd.DataFrame()\ndfn['tweet_orig'] = train_dataset['tweet_text']\ndfn['tweet_new'] = reviews_train_clean\n\ndfn.to_csv('/content/drive/My Drive/lrtext1.csv', header=True, index=False) \n\n\nreviews_train_clean = cleaningPart2(train_dataset['tweet_text'])\n  "

In [10]:
import os
import tensorflow
os.environ['KERAS_BACKEND'] = 'tensorflow'

import keras
from keras.models import Sequential
from keras.layers import*
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.regularizers import L1L2
import pandas as pd
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from keras.initializers import Constant
from termcolor import colored

max_features = 19000

# Tokenization
print(colored("Tokenizing and padding data", "yellow"))
tokenizer = Tokenizer(num_words = max_features, split = ' ', oov_token='<unw>')
tokenizer.fit_on_texts(reviews_train_clean)
train_tweets = tokenizer.texts_to_sequences(reviews_train_clean)
max_len = max([len(i) for i in train_tweets])
train_tweets = pad_sequences(train_tweets, maxlen = max_len)

print(colored("Tokenizing and padding complete", "yellow"))

## embedding code 
embeddings_index = {}
f = open(os.path.join('/content/drive/My Drive/', 'glove.twitter.27B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

num_words = min(max_features, len(word_index)) + 1
print(num_words)

embedding_dim = 100

# first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))

# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in word_index.items():
    if i > max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector
    else:
        # doesn't exist, assign a random vector
        embedding_matrix[i] = np.random.randn(embedding_dim)


# Building the model
model = Sequential()

model.add(Embedding(num_words,
                    embedding_dim,
                    embeddings_initializer=Constant(embedding_matrix),
                    input_length = train_tweets.shape[1],
                    trainable=True))

model.add(SpatialDropout1D(0.4))
model.add(LSTM(100,dropout = 0.3, recurrent_dropout=0.2))
model.add(Dense(3, init='uniform'))
model.add(BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

# Training the model
print(colored("Training the LSTM model", "green"))
history = model.fit(train_tweets, pd.get_dummies(train_dataset['sentiment']).values, epochs = 8, batch_size = 128, validation_split = 0.2)
print(colored(history, "green"))



Tokenizing and padding data
Tokenizing and padding complete
Found 1193514 word vectors.
Found 21990 unique tokens.
19001


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:77: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(3, kernel_initializer="uniform")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:78: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)`


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 530, 100)          1900100   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 530, 100)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 303       
_________________________________________________________________
batch_normalization_1 (Batch (None, 3)                 12        
_________________________________________________________________
activation_1 (Activation)    (None, 3)                 0         
Total params: 1,980,815
Trainable params: 1,980,809
Non-trainable params: 6
____________________________________________

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 17172 samples, validate on 4293 samples
Epoch 1/8
17172/17172 [==============================] - 135s 8ms/step - loss: 1.0292 - accuracy: 0.4902 - val_loss: 0.9569 - val_accuracy: 0.5295
Epoch 2/8
17172/17172 [==============================] - 134s 8ms/step - loss: 0.9224 - accuracy: 0.5701 - val_loss: 0.9117 - val_accuracy: 0.5570
Epoch 3/8
17172/17172 [==============================] - 134s 8ms/step - loss: 0.8596 - accuracy: 0.6090 - val_loss: 0.9029 - val_accuracy: 0.5770
Epoch 4/8
17172/17172 [==============================] - 133s 8ms/step - loss: 0.8112 - accuracy: 0.6377 - val_loss: 0.8932 - val_accuracy: 0.5798
Epoch 5/8
17172/17172 [==============================] - 133s 8ms/step - loss: 0.7722 - accuracy: 0.6575 - val_loss: 0.8901 - val_accuracy: 0.5903
Epoch 6/8
17172/17172 [==============================] - 133s 8ms/step - loss: 0.7427 - accuracy: 0.6746 - val_loss: 0.9002 - val_accuracy: 0.5851
Epoch 7/8
17172/17172 [==============================] - 132s 8ms/ste

In [ ]:
# Load data
test = pd.read_csv('/content/drive/My Drive/test_samples.txt')
test_data = pd.DataFrame(test, columns= ['tweet_id','tweet_text'])
td = cleaning_part2(test_data['tweet_text'])
max_len = max([len(i) for i in train_tweets])
print(max_len,"maxlen")
print(colored("Test Data loaded", "yellow"))

test_tweets = tokenizer.texts_to_sequences(td)
test_tweets = pad_sequences(test_tweets, maxlen = max_len)

result = model.predict_classes(test_tweets)
labels=[]
for res in result:
  if(res==0):
    labels.append("negative")
  if(res==1):
    labels.append("neutral")
  if(res==2):
    labels.append("positive")

ids = []
test_data['tweet_id'].apply(lambda x : ids.append(x))
data = {'tweet_id':ids , 'sentiment':labels} 
# Create DataFrame 
df = pd.DataFrame() 
df['tweet_id'] = ids
df['sentiment'] = labels
print(df)
df.to_csv('/content/drive/My Drive/test_accuracy2.csv', header=True, index=False)


530 maxlen
Test Data loaded
                tweet_id sentiment
0     264238274963451904   neutral
1     218775148495515649  positive
2     258965201766998017   neutral
3     262926411352903682   neutral
4     171874368908050432  negative
...                  ...       ...
5393  210378118865756160   neutral
5394  245177521304399872  positive
5395  259280987089932288   neutral
5396  201113950211940352   neutral
5397  237999067286876160   neutral

[5398 rows x 2 columns]
